In [30]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [76]:
ds, info = tfds.load("pop_sign", split='validation[:10%]', data_dir=".", with_info=True)

In [32]:
ds.cardinality()

<tf.Tensor: shape=(), dtype=int64, numpy=1890>

In [17]:
results = []

for example in ds:
    pose = example["pose"]
    res = tf.reduce_any(tf.math.is_nan(pose))
    results.append(res)

In [18]:
import numpy as np

np.any(results)

False

In [19]:
pose.shape

TensorShape([16, 543, 3])

In [42]:
import numpy as np
import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter("./model.tflite")
prediction_fn = interpreter.get_signature_runner("serving_default")
preds = []

for example in ds:
    pose = example["pose"]
    output = prediction_fn(inputs=pose)
    sign = np.argmax(output["outputs"])
    preds.append(sign)

In [45]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [98]:
import numpy as np
import tflite_runtime.interpreter as tflite
import pandas as pd
import random

interpreter = tflite.Interpreter("./model.tflite")
prediction_fn = interpreter.get_signature_runner("serving_default")

SOURCE_PATH = "./downloads/extracted/ZIP.kagg-comp-data_kagg-v2_4610_5087_comp_trai9p7zcoakwSxU8gbhej-Ui0WA5xI1-KSjJcJbbB6BSgE.zip"
info = pd.read_csv("train.csv")

preds = []
trues = info["sign"].to_list()
paths = info["path"].to_list()
# for path in random.sample(paths, k=100):
for path in paths[:100]:
    filepath = path.replace("train_landmark_files", "")
    filepath = SOURCE_PATH + "/" + filepath
    pose = load_relevant_data_subset(filepath)
    output = prediction_fn(inputs=pose)
    sign = np.argmax(output["outputs"])
    preds.append(sign)

In [92]:
labels = list(info.features["label"]._str2int.keys())
print(labels)
indexes = [labels.index(v) for v in labels]
print(indexes)

['blow', 'wait', 'cloud', 'bird', 'owie', 'duck', 'minemy', 'lips', 'flower', 'time', 'vacuum', 'apple', 'puzzle', 'mitten', 'there', 'dry', 'shirt', 'owl', 'yellow', 'not', 'zipper', 'clean', 'closet', 'quiet', 'have', 'brother', 'clown', 'cheek', 'cute', 'store', 'shoe', 'wet', 'see', 'empty', 'fall', 'balloon', 'frenchfries', 'finger', 'same', 'cry', 'hungry', 'orange', 'milk', 'go', 'drawer', 'TV', 'another', 'giraffe', 'wake', 'bee', 'bad', 'can', 'say', 'callonphone', 'finish', 'old', 'backyard', 'sick', 'look', 'that', 'black', 'yourself', 'open', 'alligator', 'moon', 'find', 'pizza', 'shhh', 'fast', 'jacket', 'scissors', 'now', 'man', 'sticky', 'jump', 'sleep', 'sun', 'first', 'grass', 'uncle', 'fish', 'cowboy', 'snow', 'dryer', 'green', 'bug', 'nap', 'feet', 'yucky', 'morning', 'sad', 'face', 'penny', 'gift', 'night', 'hair', 'who', 'think', 'brown', 'mad', 'bed', 'drink', 'stay', 'flag', 'tooth', 'awake', 'thankyou', 'hot', 'like', 'where', 'hesheit', 'potty', 'down', 'stuck'

In [88]:
import json
   
f = open('sign_to_prediction_index_map.json')
data = json.load(f)

In [91]:
real_labels = list(data.keys())
print(real_labels)

['TV', 'after', 'airplane', 'all', 'alligator', 'animal', 'another', 'any', 'apple', 'arm', 'aunt', 'awake', 'backyard', 'bad', 'balloon', 'bath', 'because', 'bed', 'bedroom', 'bee', 'before', 'beside', 'better', 'bird', 'black', 'blow', 'blue', 'boat', 'book', 'boy', 'brother', 'brown', 'bug', 'bye', 'callonphone', 'can', 'car', 'carrot', 'cat', 'cereal', 'chair', 'cheek', 'child', 'chin', 'chocolate', 'clean', 'close', 'closet', 'cloud', 'clown', 'cow', 'cowboy', 'cry', 'cut', 'cute', 'dad', 'dance', 'dirty', 'dog', 'doll', 'donkey', 'down', 'drawer', 'drink', 'drop', 'dry', 'dryer', 'duck', 'ear', 'elephant', 'empty', 'every', 'eye', 'face', 'fall', 'farm', 'fast', 'feet', 'find', 'fine', 'finger', 'finish', 'fireman', 'first', 'fish', 'flag', 'flower', 'food', 'for', 'frenchfries', 'frog', 'garbage', 'gift', 'giraffe', 'girl', 'give', 'glasswindow', 'go', 'goose', 'grandma', 'grandpa', 'grass', 'green', 'gum', 'hair', 'happy', 'hat', 'hate', 'have', 'haveto', 'head', 'hear', 'helic

In [93]:
reordered_indexes = [labels.index(v) for v in real_labels]
print(reordered_indexes)

[45, 246, 211, 131, 63, 119, 46, 140, 11, 184, 149, 105, 56, 50, 35, 148, 198, 100, 232, 49, 156, 121, 191, 3, 60, 0, 213, 130, 212, 224, 25, 98, 85, 127, 53, 51, 247, 202, 200, 221, 243, 27, 173, 196, 136, 21, 194, 22, 2, 26, 218, 81, 39, 170, 28, 165, 193, 242, 203, 157, 144, 112, 44, 101, 146, 15, 83, 5, 234, 245, 33, 231, 180, 91, 34, 139, 68, 87, 65, 129, 37, 54, 155, 77, 80, 103, 8, 116, 227, 36, 120, 215, 93, 47, 189, 163, 228, 43, 249, 135, 186, 78, 84, 166, 95, 125, 244, 151, 24, 137, 115, 167, 239, 238, 161, 110, 248, 128, 236, 177, 107, 40, 214, 199, 229, 69, 205, 74, 159, 154, 240, 207, 108, 145, 7, 162, 58, 160, 99, 226, 72, 147, 42, 6, 13, 208, 64, 89, 204, 223, 86, 233, 94, 114, 122, 209, 19, 71, 118, 55, 152, 62, 41, 169, 4, 17, 237, 175, 133, 92, 219, 178, 66, 172, 183, 188, 111, 153, 117, 220, 12, 23, 138, 142, 230, 217, 168, 197, 241, 90, 38, 52, 70, 32, 67, 16, 30, 206, 57, 75, 134, 174, 181, 82, 158, 102, 73, 29, 182, 113, 76, 179, 185, 195, 106, 59, 14, 97, 225, 1

In [109]:
np.sum(np.array([labels[pred] for pred in preds]) == np.array(trues[:100])) / 100

0.3